In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [2]:
data = pd.read_csv("data.csv")
data

,Month,Sunspots
0,1749-01,58.0
1,1749-02,62.6
2,1749-03,70.0
3,1749-04,55.7
4,1749-05,85.0
...,...,...
2815,1983-08,71.8
2816,1983-09,50.3
2817,1983-10,55.8
2818,1983-11,33.3


In [3]:
# 2. PREPROCESAMIENTO DE DATOS
# Convertir la columna "Month" en un número de meses desde la fecha inicial
data['Month'] = pd.to_datetime(data['Month'])
data

,Month,Sunspots
0,1749-01-01,58.0
1,1749-02-01,62.6
2,1749-03-01,70.0
3,1749-04-01,55.7
4,1749-05-01,85.0
...,...,...
2815,1983-08-01,71.8
2816,1983-09-01,50.3
2817,1983-10-01,55.8
2818,1983-11-01,33.3


In [4]:
# 3. PREPROCESAMIENTO DE DATOS
# Dividir el dataset en features (X) y target (y)
X = data[['Month']]
y = data['Sunspots']

# Normalizar los datos
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y.values.reshape(-1, 1))

### Primera Red: Feed Forward NN (Red Simple) 

In [5]:
# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. CONSTRUIR Y ENTRENAR EL MODELO
# Creando un modelo de perceptrón multicapa (MLP)
mlp = MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu', solver='adam', max_iter=1000, random_state=42)

# Entrenando el modelo
mlp.fit(X_train, y_train.ravel())

# 5. EVALUAR EL MODELO
mlp_loss = ((mlp.predict(X_test) - y_test.ravel())**2).mean()
print(f"MSE: {mlp_loss}")

MSE: 0.026854056760068266


### Segunda Red: RNN

In [15]:
# Convierte la columna "Month" en un formato de fecha
data["Month"] = pd.to_datetime(data["Month"])

# Convierte la fecha en días transcurridos desde una fecha de referencia
data["Month"] = (data["Month"] - data["Month"].min()) / np.timedelta64(1, "D")

# Ordena los datos por fecha si no están ordenados
data.sort_values(by="Month", inplace=True)

In [16]:
# Normaliza los datos de Sunspots
scaler = StandardScaler()
data["Sunspots"] = scaler.fit_transform(data["Sunspots"].values.reshape(-1, 1))

In [17]:
# Divide los datos en conjuntos de entrenamiento y prueba
X = data["Month"].values  # Utiliza ".values" para obtener una matriz NumPy
y = data["Sunspots"].values  # Utiliza ".values" para obtener una matriz NumPy

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
# Crea una serie temporal para la entrada de la RNN
window_size = 20  # Define el tamaño de la ventana (aumentado)
X_train_seq = [X_train[i:i+window_size] for i in range(len(X_train) - window_size)]
X_train_seq = np.array(X_train_seq)

X_test_seq = [X_test[i:i+window_size] for i in range(len(X_test) - window_size)]
X_test_seq = np.array(X_test_seq)

In [19]:
# Define la arquitectura de la RNN (aumentado el número de unidades)
model = keras.Sequential([
    keras.layers.SimpleRNN(100, activation="relu", input_shape=(window_size, 1)),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(1)
])

model.compile(optimizer="adam", loss="mse")

In [20]:
# Entrena el modelo
model.fit(X_train_seq, y_train[window_size:], epochs=100, batch_size=32, validation_data=(X_test_seq, y_test[window_size:]))

# Realiza predicciones
X_test_seq = X_test_seq.reshape(X_test_seq.shape[0], window_size, 1)
predictions = model.predict(X_test_seq)

Epoch 1/100
70/70 [==============================] - 1s 4ms/step - loss: 1.0037 - val_loss: 0.9796
Epoch 2/100
70/70 [==============================] - 0s 3ms/step - loss: 1.0037 - val_loss: 0.9795
Epoch 3/100
70/70 [==============================] - 0s 3ms/step - loss: 1.0034 - val_loss: 0.9791
Epoch 4/100
70/70 [==============================] - 0s 3ms/step - loss: 1.0034 - val_loss: 0.9795
Epoch 5/100
70/70 [==============================] - 0s 3ms/step - loss: 1.0033 - val_loss: 0.9791
Epoch 6/100
70/70 [==============================] - 0s 3ms/step - loss: 1.0034 - val_loss: 0.9791
Epoch 7/100
70/70 [==============================] - 0s 3ms/step - loss: 1.0039 - val_loss: 0.9791
Epoch 8/100
70/70 [==============================] - 0s 3ms/step - loss: 1.0035 - val_loss: 0.9794
Epoch 9/100
70/70 [==============================] - 0s 3ms/step - loss: 1.0034 - val_loss: 0.9791
Epoch 10/100
70/70 [==============================] - 0s 3ms/step - loss: 1.0033 - val_loss: 0.9792
Epoch 11/

In [21]:
# Calcula el MSE
mse = mean_squared_error(y_test[window_size:], predictions)

print("Mean Squared Error (MSE):", mse)

Mean Squared Error (MSE): 0.9793040581314121


### Tercera Red: RNN

In [22]:
# 1. CARGANDO LOS DATOS
data_lstm = pd.read_csv("data.csv")

# 2. PREPROCESAMIENTO DE DATOS
# Convertir la columna "Month" en un formato de fecha
data_lstm['Month'] = pd.to_datetime(data_lstm['Month'])

In [23]:

# Escalar la columna "Sunspots"
scaler = MinMaxScaler()
data_lstm["Sunspots"] = scaler.fit_transform(data_lstm["Sunspots"].values.reshape(-1, 1))

# Crear secuencias de datos
SEQUENCE_LENGTH = 10  # Longitud de la secuencia

X, y = [], []
for i in range(len(data_lstm) - SEQUENCE_LENGTH):
    X.append(data_lstm["Sunspots"].iloc[i : i + SEQUENCE_LENGTH].values)
    y.append(data_lstm["Sunspots"].iloc[i + SEQUENCE_LENGTH])

# Convertir a arrays numpy para el entrenamiento
X = np.array(X)
y = np.array(y)

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. CONSTRUIR Y ENTRENAR EL MODELO
# Creando un modelo de perceptrón multicapa (MLP) como aproximación de LSTM
mlp_lstm = MLPRegressor(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', max_iter=1000, random_state=42)

# Entrenar el modelo
mlp_lstm.fit(X_train, y_train)

# 4. EVALUAR EL MODELO
mlp_lstm_loss = ((mlp_lstm.predict(X_test) - y_test)**2).mean()
print(f"MSE: {mlp_lstm_loss}")

MSE: 0.004208807754181249
